# Modelo de Intercambio Aleatorio de Activos (RAE)

## Sistema económico cerrado con conservación de dinero

El modelo utilizado es el de [Dragulescu & Yakovenko (2000)](https://doi.org/10.1007/s100510070114)

### Descripción del modelo

Los agentes económicos interaccionan por pares mediante transacciones monetarias. 

Solo se puede dar o recibir dinero en las transacciones.

Los agentes perdedores se etiquetan con el indice $i$ mientras que los ganadores con el indice $j$.

Sean $m$ y $m'$ la cantidad de dinero que posee un agente, antes y después de la transacción la cual se da según:

$$ m_i \rightarrow m'_i = m_i - \Delta m$$
$$ m_j \rightarrow m'_j = m_j + \Delta m$$

donde $\Delta m$ es la cantidad de dinero que el agente perdedor le transfirió al agente ganador después de la transacción.


Algunos supuestos:
- El dinero se conserva, global y localmente, $m_i + m_j = m'_i + m'_j $.
- El balance de los ganadores no puede ser negativo, $m_j  \ge 0 \ \forall j $.
- Una transacción se da solamente cuando el agente "perdedor" tiene suficiente dinero para pagar, $m_i  \ge \Delta m$.
- No hay crédito ni deuda.


La mecánica estadística predice que el sistema económico alcanzara la distribución estacionaria de Boltzmann-Gibbs,

$$ P(m) = \frac{1}{T} e^{-m/T} $$

donde 

$$ T = \langle m \rangle = \frac{M}{N} $$

es la _"Temperatura del dinero"_, y es igual a la cantidad promedio de dinero. $M$ es la cantidad total de dinero en el sistema, y $N$ es el numero total de agentes en el sistema. 


## Cómo se realizan las simulaciones

Las simulaciones se realizan en el siguiente orden:

- Inicialmente todos los agentes tienen la misma cantidad de dinero.
- Se elige un par $(i, j)$ de agentes al azar, y e realiza la transacción, es decir, se transfiere la cantidad $\Delta m$.
- Se repite el paso anterior varias veces.



En este trabajo se realizan las simulaciones con cada una de las siguientes reglas de intercambio:

- **Regla 1**: Intercambio de una pequeña cantidad constante de dinero, $\Delta m = 1$.
- **Regla 2**: Intercambio de una fracción aleatoria $0 \le \nu \le 1$ del dinero promedio del par, $\Delta m = \nu \left( m_i + m_j \right)/2$
- **Regla 3**: Intercambio de una fracción aleatoria $0 \le \nu \le 1$ del dinero promedio del sistema,  $\Delta m = \nu T$

Observese que estas reglas tienen simetría de inversión temporal, ya que son invariantes ante una permutación de los índices $i$ y $j$.

### Paquetes a utilizar 

In [ ]:
using Random, Distributions, CairoMakie, LaTeXStrings

### Condiciones iniciales

Los parámetros para las simulaciones son: $N = 5 \times 10^{5}$ y $M = 5 \times 10^{6}$. Por lo tanto, al principio de las simulaciones, los agentes contarán con la misma cantidad de dinero $\left\langle m \right\rangle = \frac{M}{N} = 10$.

In [59]:
N_exp = 5
N = 5 * 10^N_exp     

M_exp = 6
M = 5 * 10^M_exp

T = M/N 

10.0

#### Reglas de intercambio 

In [60]:
exchange_rules = ["small_constant", "random_pair_average", "random_system_average"]

3-element Vector{String}:
 "small_constant"
 "random_pair_average"
 "random_system_average"

In [61]:
mutable struct Agent
  money::Float64
end

In [62]:
p = Agent(3)

Agent(3.0)

In [63]:
p.money

3.0

In [64]:
# Initial conditions
N_exp = 5
M_exp = 6
T_exp = 7
million = 1_000_000
N = Int(5 * 10^N_exp)  # numero de agentes   
M = Int(5 * 10^M_exp) # Dinero total en el sistema 
T = Int(4 * 10^T_exp) # tiempo
M_prom = Int(M/N) # temperatura del dinero


10

In [65]:
function sim_01(N, M, M_prom, rule, T)
 
  Agentes = [Agent(M_prom) for i in 1:N]
  exchange_rules = ["small_constant", "random_pair_average", "random_system_average"]
  transaction_type = exchange_rules[rule]
  system_avg = M_prom
  for t in 1:T
    agent_i = rand(1:N)
    agent_j = rand(1:N)

    coin_flit = rand(1:2)
    loser_index = [agent_i, agent_j][coin_flit]
    winner_index = [agent_i, agent_j][2-coin_flit+1]

    delta_m = 0.0
    
    if transaction_type == "small_constant"
    delta_m += 1.0
    elseif transaction_type == "random_pair_average"
    avg = (Agentes[loser_index].money + Agentes[winner_index].money) / 2
    delta_m += rand(Uniform()) * avg
    elseif transaction_type == "random_system_average"
    delta_m += rand(Uniform()) * system_avg
    elseif transaction_type == "proportional"
    delta_m += 0.5 * lista.money
    end
    delta_m

    if Agentes[loser_index].money >= delta_m
      Agentes[loser_index].money -= delta_m
      Agentes[winner_index].money += delta_m
    end
  end

  wealth = [0.0 for i in 1:N]
  for i in 1:N
    wealth[i] = Agentes[i].money
  end

return wealth
end

sim_01 (generic function with 1 method)

In [66]:
sim = simulation(5, 6, 2, 7)

UndefVarError: UndefVarError: `simulation` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [67]:
length(sim)

500000

In [68]:
max_val_sim = maximum(sim)
max_val_sim_int = Int(floor(max_val_sim))
m_values = range(0, stop = max_val_sim, length=max_val_sim_int)
P_values = length(sim) * (1 / 10) * exp.(-m_values / 10)  

127-element Vector{Float64}:
 50000.0
 45200.69195277662
 40862.051060196114
 36939.85965061108
 33394.14433692148
 30188.76862399496
 27291.06462013689
 24671.500099162567
 22303.377519902904
 20162.561935672285
     ⋮
     0.33699954538331506
     0.30465225278193997
     0.2754098526143182
     0.2489743181755879
     0.22507622920014703
     0.20347202603936798
     0.18394152740025607
     0.166285686346844
     0.1503245616943947

In [69]:
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Distribución del dinero",
    xlabel = "Cantidad de Dinero",
    ylabel = "Agentes",
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = "Simulación")
lines!(m_values, P_values, color=:red, label = L"P(m)=\frac{1}{T}e^{-m/T}")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

En las simulaciones observamos que una distribución inicial estrecha, en la que todos los agentes tienen la misma cantidad de dinero, es inestable y evoluciona con el tiempo hacia una distribución amplia y sesgada, en la que muchos agentes tienen saldos monetarios bajos y pocos agentes tienen saldos monetarios altos. Finalmente, la distribución de dinero alcanza el equilibrio estadístico en la forma exponencial (la distribución de Boltzmann-Gibbs), de acuerdo con los principios generales de la física estadística y el principio de máxima entropía. 

Sin embargo, si una regla para transferencias de dinero no tiene simetría de inversión temporal , se pueden obtener otras distribuciones.

## Sistema económico cerrado con conservación de dinero 2.0: se rompe la simetría de inversión temporal

En este modelo de intercambio, la regla de transferencia es $\Delta m = \alpha m_i$, es decir, el intercambio es proporcional al saldo monetario del agente. 
Con esta regla se rompe la simetría de inversión temporal, por lo que el sistema puede tener una distribución estacionaria diferente a la de Boltzmann-Gibbs, o no tener ninguna distribución estacionaria.

Los parámetros para las simulaciones serán: $N = 5 \times 10^{5}$ y $M = 5 \times 10^{6}$. Por lo tanto, al principio de las simulaciones, los agentes contarán con la misma cantidad de dinero $\left\langle m \right\rangle = \frac{M}{N} = 10$.

Primero realizamos una simulación de intercambio aleatorio multiplicativo con $\alpha = 1/3$

In [70]:
# Initial conditions
N_exp = 5
M_exp = 6
T_exp = 7
million = 1_000_000
N = Int(5 * 10^N_exp)  # numero de agentes   
M = Int(5 * 10^M_exp) # Dinero total en el sistema 
T = Int(4 * 10^T_exp) # tiempo
M_prom = Int(M/N) # temperatura del dinero
α = 1/3

0.3333333333333333

In [71]:
Int(M_prom)

10

In [72]:
function sim_02(N, T, M_prom, α)
  Agentes = [Agent(M_prom) for i in 1:N]
  
  for t in 1:T
    agent_i = rand(1:N)
    agent_j = rand(1:N)

    coin_flit = rand(1:2)
    loser_index = [agent_i, agent_j][coin_flit]
    winner_index = [agent_i, agent_j][2-coin_flit+1]

    delta_m = α *  Agentes[loser_index].money

    if Agentes[loser_index].money >= delta_m
      Agentes[loser_index].money -= delta_m
      Agentes[winner_index].money += delta_m
    end
  end

  wealth = [0.0 for i in 1:N]
  for i in 1:N
    wealth[i] = Agentes[i].money
  end

return wealth
end

sim_02 (generic function with 1 method)

In [73]:
sim = sim_02(N, T, M_prom, α)

500000-element Vector{Float64}:
 27.068782618769536
  1.700542700718863
  0.9571380555264384
 12.575613299564559
  9.169415400420649
  3.9386786655012287
  3.4100976763846917
  0.788883152292948
  9.025445797260762
  8.078099718716768
  ⋮
 21.304959588340296
 12.785254491312193
  9.288256706675174
 12.98946678340528
  1.5467838501156912
  3.290359137405299
  8.855647827298178
  3.1096940757375853
 10.928533985288402

In [74]:
max_val_sim = maximum(sim)
max_val_sim_int = Int(floor(max_val_sim))
m_values = range(0, stop = max_val_sim, length=max_val_sim_int)
P_values = length(sim) * (1 / M_prom) * exp.(-m_values / M_prom)  

74-element Vector{Float64}:
 50000.0
 45177.12181770544
 40819.44671463594
 36882.10233516887
 33324.54460178005
 30110.14021988351
 27205.789453237383
 24581.585285515022
 22210.505458320556
 20068.134214467187
     ⋮
    68.48982582197304
    61.88346408865407
    55.9143359126945
    50.520975297678035
    45.64784510744976
    41.24476518270015
    37.26639562003001
    33.671769892658006
    30.42387300516715

In [75]:
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $\alpha = 1/3$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Podemos observar que la distribución estacionaria tiene una forma diferente a la distribución B-G: todavía tiene un extremo exponencial en el extremo superior, pero cae a cero en el extremo inferior. 

Esto último se cumple para simulaciones con $\alpha < 1/2$. 

Por otra parte, si realizamos una simulación de intercambio aleatorio multiplicativo con $\alpha = 1/2$, tenemos que

In [76]:
α=1/2
sim = sim_02(N, T, M_prom, α)

500000-element Vector{Float64}:
 10.446043358332297
  3.097629646506975
 10.428477761634609
  0.5963769088259775
 11.885839688913894
  7.24455504055737
  0.11857661454049383
 16.833849294128715
  1.988509861476634
 27.64288529341392
  ⋮
  4.800216652810429
  2.5382751592947925
  1.1798103087448202
  9.355600796028932
  9.53677338771006
  6.6045399385683945
 15.933669258898963
  6.436329048834903
 45.31475564751668

In [77]:
max_val_sim = maximum(sim)
max_val_sim_int = Int(floor(max_val_sim))
m_values = range(0, stop = max_val_sim, length=max_val_sim_int)
P_values = length(sim) * (1 / M_prom) * exp.(-m_values / M_prom) 

173-element Vector{Float64}:
 50000.0
 45201.49954622435
 40863.51122454642
 36941.83968146938
 33396.53099197262
 30191.465609582356
 27293.990381027706
 24674.585876453515
 22306.56564595575
 20165.804338469854
     ⋮
     0.0032580330546711565
     0.0029453595928460507
     0.0026626934059899624
     0.0024071546956518025
     0.0021761400376639323
     0.0019672958584997353
     0.0017784944571053005
     0.0016078123279161531
     0.0014535105642143216

In [78]:
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $\alpha = 1/2$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Modelo de intercambio aleatorio con ahorro: Simulaciones

Otro ejemplo de tipo similar al anterior es uno donde los agentes guardan una fracción $\lambda$ de su dinero e intercambian una fracción aleatoria $\epsilon$ de su dinero total restante,

$$
m_i \longrightarrow m'_i = \lambda m_{i} + \epsilon \left( 1-\lambda \right)\left( m_i + m_j \right) \\
m_j \longrightarrow m'_j = \lambda m_{j} + \left( 1-\epsilon \right) \left( 1-\lambda \right)\left( m_i + m_j \right)
$$

de donde se deduce que la regla de intercambio es

$$
\Delta m = m_i - m'_i = \left( 1-\lambda \right)\left[ m_i - \epsilon \left( m_i+m_j \right) \right]
$$

En esta regla de intercambio se rompe la simetría de inversión temporal, por lo que la distribución estacionaria puede ser diferente a la distribución B-G.

Los parámetros para una simulación serán: $\lambda = 1/4$, $N = 5 \times 10^{5}$ y $M = 5 \times 10^{6}$. 

Por lo tanto, al principio de las simulaciones, los agentes contarán con la misma cantidad de dinero $\left\langle m \right\rangle = \frac{M}{N} = 10$.

In [79]:
# Initial conditions
N_exp = 5
M_exp = 6
T_exp = 7
million = 1_000_000
N = Int(5 * 10^N_exp)  # numero de agentes   
M = Int(5 * 10^M_exp) # Dinero total en el sistema 
T = Int(4 * 10^T_exp) # tiempo
M_prom = Int(M/N) # temperatura del dinero
λ = 1/4

0.25

In [80]:
function sim_03(N, T, M_prom, λ)
  Agentes = [Agent(M_prom) for i in 1:N]
  
  for t in 1:T
    ϵ = rand(Uniform(0.0, 1.0))
    agent_i = rand(1:N)
    agent_j = rand(1:N)

    coin_flit = rand(1:2)
    loser_index = [agent_i, agent_j][coin_flit]
    winner_index = [agent_i, agent_j][2-coin_flit+1]

    delta_m = (1 - λ) *  ( Agentes[loser_index].money - ϵ * (Agentes[loser_index].money + Agentes[winner_index].money))

    if Agentes[loser_index].money >= delta_m
      Agentes[loser_index].money -= delta_m
      Agentes[winner_index].money += delta_m
    end
  end

  wealth = [0.0 for i in 1:N]
  for i in 1:N
    wealth[i] = Agentes[i].money
  end

return wealth
end

sim_03 (generic function with 1 method)

In [81]:
sim = sim_03(N, T, M_prom, λ)

UndefVarError: UndefVarError: `Uniform` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [82]:
# parameters B-G distribution
max_val_sim = maximum(sim)
max_val_sim_int = Int(floor(max_val_sim))
m_values = range(0, stop = max_val_sim, length=max_val_sim_int)
P_values = length(sim) * (1 / M_prom) * exp.(-m_values / M_prom) 

# Graph
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $\lambda = 1/4$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [83]:
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
    yscale = log
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $\text{tax rate} = 0.4$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Se puede observar que la forma de la distribución estacionaria tiene una forma cualitativamente similar a la que tiene la distribución estacionaria del modelo de intercambio multiplicativo.

### Modelo de intercambio aleatorio con impuestos y subsidios.

Otro ejemplo interesante cuya distribución no es de B-G ocurre en un modelo con impuestos y subsidios. 

Supongamos que hay un agente especial ("gobierno"), el cual recolecta una fracción $f$ ("impuesto") de toda transacción en el sistema, de tal manera que cada agente recibe un subsidio $\delta m$ con una frecuencia $1 / \tau_s$.

A continuación se muestran los resultados de una simulación. 

Los parámetros son: $f = 0.40$, $\tau_s = 4000$, $N = 5 \times 10^{4}$ y $M = 5 \times 10^{5}$. 

Por lo tanto, al principio de las simulaciones, los agentes cuentan con la misma cantidad de dinero $\left\langle m \right\rangle = \frac{M}{N} = 10$.

In [84]:
# Initial conditions
N_exp = 4
M_exp = 5
T_exp = 6
million = 1_000_000
N = Int(5 * 10^N_exp)  # numero de agentes   
M = Int(5 * 10^M_exp) # Dinero total en el sistema 
T = Int(4 * 10^T_exp) # tiempo
M_prom = Int(M/N) # temperatura del dinero
tax_rate = 0.4 #f 
τₛ = 4000

4000

In [85]:
mutable struct State
  money::Float64
  tax_rate::Float64
end

In [86]:
function collect_taxes(gob::State, from_amount)
  gob.money += (gob.tax_rate * from_amount)
end

collect_taxes (generic function with 1 method)

In [87]:
function subsidize(gob::State, to_agents)
  dm = gob.money / length(to_agents)
  for i in length(to_agents)
    gob.money -= dm
    to_agents[i].money += dm
  end
end

subsidize (generic function with 1 method)

In [88]:
government = State(0.0, tax_rate)
government.tax_rate

0.4

In [89]:
function sim_04(N, M, M_prom, rule, T, tax_rate, τₛ)
	
  Agentes = [Agent(M_prom) for i in 1:N]
  government = State(0.0, tax_rate)

  exchange_rules = ["small_constant", "random_pair_average", "random_system_average"]
  transaction_type = exchange_rules[rule]
  system_avg = M_prom
  for t in 1:T
    agent_i = rand(1:N)
    agent_j = rand(1:N)

    coin_flit = rand(1:2)
    loser_index = [agent_i, agent_j][coin_flit]
    winner_index = [agent_i, agent_j][2-coin_flit+1]

    delta_m = 0.0
    
    if transaction_type == "small_constant"
    delta_m += 1.0
    elseif transaction_type == "random_pair_average"
    avg = (Agentes[loser_index].money + Agentes[winner_index].money) / 2
    delta_m += rand(Uniform()) * avg
    elseif transaction_type == "random_system_average"
    delta_m += rand(Uniform()) * system_avg
    elseif transaction_type == "proportional"
    delta_m += 0.5 * lista.money
    end
    delta_m

    if Agentes[loser_index].money >= delta_m
      Agentes[loser_index].money -= delta_m
      government.money += (government.tax_rate * delta_m)
      delta_m *= (1- government.tax_rate)
      Agentes[winner_index].money += delta_m
    end

    if t % τₛ == 0
      dm = government.money / N
      for i in 1:N
      government.money -= dm
      Agentes[i].money += dm 
      end
    end
  end

  wealth = [0.0 for i in 1:N]
  for i in 1:N
    wealth[i] = Agentes[i].money
  end

return wealth
end

sim_04 (generic function with 1 method)

In [90]:
sim = sim_04(N, M, M_prom, 3, T, tax_rate, τₛ)

UndefVarError: UndefVarError: `Uniform` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [91]:
# parameters B-G distribution
max_val_sim = maximum(sim)
max_val_sim_int = Int(floor(max_val_sim))
m_values = range(0, stop = max_val_sim, length=max_val_sim_int)
P_values = length(sim) * (1 / M_prom) * exp.(-m_values / M_prom) 

# Graph
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
    #yscale = log
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $\text{tax rate} = 0.4$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [92]:
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
    yscale = log
	)
stephist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $\text{tax rate} = 0.4$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Se puede observar que la distribución estacionaria es una distribución B-G desplazada. La población con bajos niveles económicos se ve disminuida porque el gobierno, actuando como una fuerza externa, “expulsa” a esa población y empuja al sistema fuera del equilibrio termodinámico. 

Estos ejemplos muestran que la distribución de Boltzmann-Gibbs no es completamente universal, es decir, no se cumple para cualquier modelo de intercambio que conserve dinero. Sin embargo, es universal en un sentido limitado: para una amplia clase de modelos que tienen simetría de inversión temporal, la distribución estacionaria es exponencial y no depende de los detalles de un modelo. 

Por el contrario, cuando se rompe la simetría de inversión temporal, la distribución puede depender de los detalles del modelo. La diferencia entre estas dos clases de modelos puede ser bastante sutil.

A diferencia de lo que ocurre en la física, en economía no existe un requisito fundamental de que las interacciones tengan simetría de inversión temporal. Sin embargo, en ausencia de un conocimiento detallado de la dinámica microscópica real del intercambio económico, la distribución semiuniversal de Boltzmann-Gibbs parece ser un punto de partida natural.

## Modelos con deuda

Ahora analicemos qué sucede si se permite a los agentes endeudarse. La deuda puede considerarse como dinero negativo. Ahora bien, cuando un perdedor no tiene suficiente dinero para pagar, puede pedir prestada la cantidad necesaria a un depósito o "reservorio", y su saldo se vuelve negativo. No se viola la ley de conservación: la suma del dinero positivo del ganador y el dinero negativo del perdedor permanece constante. Cuando un agente con un saldo negativo recibe dinero como ganador, utiliza este dinero para pagar la deuda hasta que su saldo se vuelve positivo. 

Suponemos, para simplificar, que el "reservorio" no cobra intereses por el dinero prestado. Sin embargo, como no es sensato permitir una deuda ilimitada, ponemos un límite $m_d$ a la deuda máxima de un agente: $m_{i} > -m_{d}$. Esta nueva condición a la frontera $P\left( m < -m_{d} \right) = 0$ reemplaza a la vieja condición $P\left( m < 0 \right) = 0$. De nuevo $P\left( m \right)$ está dada por la ley de Boltzmann-Gibbs, 

$$
P\left( m  \right) = \frac{1}{T}\exp \left[ -\left( m+m_d \right)/T \right]
$$ 

(así se cumple la condición de normalización $\int_{-m_d}^{\infty }P\left( m  \right) dm = 1$), pero ahora,

$$
\left\langle m \right\rangle = \int_{-m_d}^{\infty }m P\left( m  \right) dm = T-m_{d}
$$

por lo que ahora se tiene una temperatura de dinero $T = M/N + m_{d}$ más alta.

### Descripción del modelo

Los agentes económicos interaccionan a pares mediante transacciones monetarias. Solo se puede recibir o dar dinero en las transacciones.

Sean $i$ ("perdedor") y $j$ ("ganador") un par de agentes económicos que interaccionan. Sean $m$ y $m'$ la cantidad de dinero que posee un agente, antes y después de la transacción, la cual se da según

$$
m_i \longrightarrow m'_i = m_i - \Delta m \\
m_j \longrightarrow m'_j = m_j + \Delta m
$$

donde $\Delta m$ es el dinero que el "perdedor" transfirió al "ganador" después de la transacción.

Suposiciones y restricciones:

* El dinero se conserva, global y localmente, $m_i + m_j = m'_i + m'_j$
* El balance de los ganadores no puede ser menor a la deuda máxima, $m_i \ge -m_{d} \forall i$
* Una transacción se da solamente cuando el agente "perdedor" tiene el suficiente dinero para pagar, $m_{i} \ge \Delta m - m_{d}$

### Cómo se realizan las simulaciones

Los pasos para realizar una simulación son los siguientes:
* Inicialmente todos los agentes tienen la misma cantidad de dinero.
* Se elige un par ($i,j$) de agentes al azar, y se realiza la transacción, es decir, se transfiere la cantidad $\Delta m$.
* Se repite el paso anterior muchas veces.

En este trabajo se realizan simulaciones con cada una de las siguientes reglas de intercambio:
* Regla 1: Intercambio de una pequeña cantidad constante, $\Delta m = 1$.
* Regla 2: Intercambio de una fracción aleatoria $0 \le \nu \le 1$ del dinero promedio del par, $\Delta m = \nu \left[ \left( m_i + m_j \right)/2 + m_d \right] $
* Regla 3: Intercambio de una fracción aleatoria $0 \le \nu \le 1$ del dinero promedio del sistema,  $\Delta m = \nu M/N$

Nótese que el modelo anterior se generaliza al modelo con deuda si hacemos el cambio de variable $m \longrightarrow m + m_{d}$.

Los parámetros para las simulaciones son: $N = 5 \times 10^{4}$ y $M = 5 \times 10^{5}$. Por lo tanto, al principio de las simulaciones, los agentes contarán con la misma cantidad de dinero $\left\langle m \right\rangle = \frac{M}{N} = 10$.

Primero realicemos una simulación para deuda máxima $m_d = 8$. La temperatura de dinero del sistema de agentes sera, por lo tanto, $T = M/N + m_d = 18$.

In [93]:
# Initial conditions
N_exp = 4
M_exp = 5
T_exp = 5
million = 1_000_000
N = Int(5 * 10^N_exp)  # numero de agentes   
M = Int(5 * 10^M_exp) # Dinero total en el sistema 
T = Int(4 * 10^T_exp) # tiempo
M_avg = Int(M/N) # temperatura del dinero

m_d = 8 #deuda maxima

M_prom = M_avg + m_d # temperatura del dinero con deuda 


18

In [94]:
mutable struct Agent_debt
  money::Float64
  max_debt::Float64
end

In [95]:
Agent_debt(10, 8)

Agent_debt(10.0, 8.0)

In [96]:
function sim_05(N, M, M_avg, m_d, rule, T)
	
  Agentes = [Agent_debt(M_avg, m_d) for i in 1:N] # money = M_avg
  exchange_rules = ["small_constant", "random_pair_average", "random_system_average"]
  transaction_type = exchange_rules[rule]
  system_avg = M_avg

  for t in 1:T
    agent_i = rand(1:N)
    agent_j = rand(1:N)

    coin_flit = rand(1:2)
    loser_index = [agent_i, agent_j][coin_flit]
    winner_index = [agent_i, agent_j][2-coin_flit+1]

    delta_m = 0.0
    
    if transaction_type == "small_constant"
    delta_m += 1.0
    elseif transaction_type == "random_pair_average"
    avg = ( (Agentes[loser_index].money + Agentes[winner_index].money) / 2 ) + Agentes[loser_index].max_debt 
    delta_m += rand(Uniform()) * avg
    elseif transaction_type == "random_system_average"
    delta_m += rand(Uniform()) * system_avg
    elseif transaction_type == "proportional"
    delta_m += 0.5 * ( lista.money + Agentes[loser_index].max_debt )
    end
    delta_m

    if (Agentes[loser_index].money + Agentes[loser_index].max_debt) >= delta_m
      Agentes[loser_index].money -= delta_m
      Agentes[winner_index].money += delta_m
    end
  end

  wealth = [0.0 for i in 1:N]
  for i in 1:N
    wealth[i] = Agentes[i].money
  end

return wealth
end

sim_05 (generic function with 1 method)

In [97]:
sim = sim_05(N, M, M_avg, m_d, 2, T)

UndefVarError: UndefVarError: `Uniform` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [98]:
minimum(sim)

1.703736382327423e-5

In [99]:
Int(floor(minimum(sim)))

0

In [100]:
m_d

8

In [101]:
max_val_sim = maximum(sim) + m_d
max_val_sim_int = Int(floor(max_val_sim) )
m_values = range(-m_d, stop = max_val_sim, length=max_val_sim_int)
P_values = length(sim) * (1 / (M_prom)) * exp.(-(m_values .+ m_d)/ (M_prom)) 

# Graph
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
    #yscale = log
	)
hist!(ax, sim, bins = max_val_sim_int, color = :black, label = L"Simulation with $m_d = 8$")
lines!(m_values, P_values, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Luego realicemos una simulación sin deuda, $m_d = 0$, para comparar. La temperatura de dinero del sistema de agentes sera, por lo tanto, $T = M/N + m_d = 10$.

In [102]:
# Initial conditions
N_exp = 4
M_exp = 5
T_exp = 5
million = 1_000_000
N = Int(5 * 10^N_exp)  # numero de agentes   
M = Int(5 * 10^M_exp) # Dinero total en el sistema 
T = Int(4 * 10^T_exp) # tiempo
M_avg = Int(M/N) # temperatura del dinero

m_d = 0 #deuda maxima

M_prom = M_avg + m_d # temperatura del dinero con deuda 

10

In [103]:
sim_2 = sim_05(N, M, M_avg, m_d, 2, T)

UndefVarError: UndefVarError: `Uniform` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [104]:
max_val_sim_2 = maximum(sim_2) + m_d
max_val_sim_int_2 = Int(floor(max_val_sim_2))
m_values_2 = range(-m_d, stop = max_val_sim_2, length=max_val_sim_int_2)
P_values_2 = length(sim) * (1 / (M_prom)) * exp.(-(m_values_2 .+ m_d)/ (M_prom)) 

# Graph
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
    #yscale = log
	)
hist!(ax, sim_2, bins = max_val_sim_int_2, color = :black, label = L"Simulation with $m_d = 0$")
lines!(m_values_2, P_values_2, color=:red, label = L"B-G Distribution $P(m)=\frac{1}{T}e^{-m/T}$")
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `sim_2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Por último, si comparamos las distribuciones B-G para ambos casos

In [105]:
# Graph
f = Figure(size = (800, 500))
ax = Axis(f[1, 1],
	title = "Money Distribution with $(N/million) million agents, $(M/million ) million money, average money $M_prom and $(T/million) million step time",
    xlabel = L"Money $m$",
    ylabel = L"\text{Distribution of Agents}",
    #yscale = log
	)
lines!(m_values, P_values, color=:blue, label = L"B-G Distribution $m_d = 8$")
vlines!(-8, color=:gray, linestyle=:dash )
lines!(m_values_2, P_values_2, color=:tomato, label = L"B-G Distribution $m_d = 0$")
vlines!(0, color=:gray, linestyle=:dash )
axislegend(position=:rt)
f

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Citando el artículo, 

> vemos que la temperatura más alta hace que la distribución del dinero sea más amplia, lo que significa que la deuda aumenta la desigualdad entre los agentes.

Para probar o refutar este enunciado, vamos a calcular el coeficiente Gini, es decir, el grado de desigualdad presente en la distribución de dinero B-G con deuda.

El coeficiente Gini de una distribución $P\left( m \right)$ se define en referencia a la curva de Lorentz, la cual se define como

$$
L\left( F\left( m \right) \right) = \frac{1}{\left\langle m \right\rangle}\int_{m_{min}}^{m}m'P\left( m' \right)dm'
$$

donde $F\left( m \right) = \int_{m_{min}}^{m}P\left( m' \right)dm'$ es la distribución acumulada de $P\left( m \right)$ y $\left\langle m \right\rangle = \int_{m_{min}}^{\infty} m' P\left( m' \right)dm'$ es la media de $P\left( m \right)$. Intuitivamente, esta integral representa la parte de algún activo que está en manos de la parte $100F$% inferior de la población. El coeficiente de Gini se obtiene entonces mediante el doble de la diferencia entre el área bajo la curva de Lorenz de una distribución perfectamente igualitaria (una línea recta con una pendiente de 1) y la curva de Lorenz de la distribución en cuestión,

$$
G = 1-2\int_{0}^{1}L\left( F \right)dF
$$

En el caso particular de la distribución B-G con deuda tenemos que $m_{min} = -m_{d}$, por lo que $F\left( m \right) = \int_{-m_{d}}^{m}\frac{1}{T}\exp \left[ -\left( m'+m_d \right)/T \right]dm' = 1 - \exp\left( -\frac{m+m_d}{T} \right) = 1 - \exp\left( -\frac{m+m_d}{\left\langle m \right\rangle+m_d} \right)$, y 

$$
L = \frac{1}{\left\langle m \right\rangle}\int_{-m_d}^{m}m'\frac{1}{T}\exp \left[ -\left( m'+m_{d} \right)/T \right]dm' = \frac{\left\langle m \right\rangle-\left( m + T \right)\exp \left[ -\left( m+m_{d} \right)/T \right]}{\left\langle m \right\rangle} \\
= \frac{\left\langle m \right\rangle-\left( m+T \right)\left( 1-F \right)}{\left\langle m \right\rangle} = \frac{\left\langle m \right\rangle- \left[ T \ln \frac{1}{1-F}-m_{d}+T \right] \left( 1-F \right)}{\left\langle m \right\rangle} \\
= \frac{\left\langle m \right\rangle - \left[ T \ln \frac{1}{1-F} + \left\langle m \right\rangle \right] \left( 1-F \right)}{\left\langle m \right\rangle} = 1 - \left( 1-F \right)\left( 1+ \frac{\left\langle m \right\rangle + m_d}{\left\langle m \right\rangle} \ln \frac{1}{1-F} \right)
$$

Grafiquemos la curvas de Lorentz correspondientes a las simulaciones,

In [106]:
M_avg

space = range(0.0, stop= 1, step=50)
L_a = log.(1 ./ (1 .- space))
L_a = L_a .* (M_avg + ) 

Base.Meta.ParseError: ParseError:
# Error @ /Users/ernesxavier/Documents/Julia Programming Models/Modelado_RAE/Julia/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y153sZmlsZQ==.jl:5:23
L_a = log.(1 ./ (1 .- space))
L_a = L_a .* (M_avg + ) 
#                     ╙ ── unexpected `)`

# Modelo más realista 

### Descripción del modelo

Para que el modelo sea un poco más realista, introducimos empresas. Un agente a la vez se convierte en una empresa. La empresa toma prestado capital $K$ de otro agente y lo devuelve con un interés $rK$, contrata $L$ agentes y les paga un salario $W$, fabrica $q$ artículos de un producto y los vende a $q$ agentes a un precio $p$. Todos estos agentes son seleccionados aleatoriamente. La empresa recibe la ganancia
$$
\pi = pq - LW - rK
$$

El resultado neto es un intercambio de dinero entre muchos cuerpos que aún satisface la ley de conservación.

Los parámetros del modelo que maximizan la ganancia se obtienen de la siguiente forma:

1) Se establece la curva de oferta-demanda agregada, $p\left( q \right) = V/q^{\eta}$, donde $q$ es la cantidad de productos que la gente compraría a un precio $p$. $V = 100$ y $\eta = 1/2$ son constantes.
2) Se establece la función de producción de la firma, $Q\left( L,K \right) = L^{\beta}K^{1-\beta}$, con $\beta = 8/10$.
3) Se sustituyen las expresiones de los pasos anteriores en la ecuación de la ganacia, y se obtiene la ecuación $F\left( L,K \right) = V Q^{1-\eta} - LW - rK = V \left(L^{\beta}K^{1-\beta}\right)^{1-\eta} - LW - rK$
4) Se resuelve $\nabla F\left( L,K \right) = 0$ para maximizar la ganancia. La solución es 
$$
L_{critico} = \left( \frac{V \beta\left( 1-\eta \right)}{W} \right)^{1/\eta}\left( \frac{W \left( 1-\beta \right)}{r\beta} \right)^{\left(1-\beta\right)\left(1-\eta\right)/\eta}\\
K_{critico} = \frac{1-\beta}{\beta}\cdot\frac{W}{r}L_{critico}
$$

Una vez se establecen los valores de los parámetros $W$ y $r$, se pueden encontrar los valores de $L_{critico}$, $K_{critico}$, $Q_{c} = Q \left( L_{critico},K_{critico}  \right)$, $R_{c} = R \left( Q_c \right)$ y $F_{max}$, en ese orden. Por ejemplo, si $W = 10$ y $r=0.15$, entonces
$$
L_{critico} = \simeq 28\\
K_{critico} = 468.1\\
Q_{c} \simeq 49\\
R_{c} = 14.242\\
F_{max} = 351.075
$$

Es decir, con aquellos valores de salario y tasa de interés, la empresa obtendrá una ganancia máxima de 351.1$ si contrata a 28 trabajadores, toma prestado 468.1$ de capital, fabrica 49 artículos de su producto y los vende a 14.2$ cada uno.